In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install d2l==0.17.6

In [ ]:
import os
path = '/content/drive/MyDrive'
os.chdir(path)

!source venv_d2l/bin/activate

path = '/content/drive/MyDrive/d2l-zh'
os.chdir(path)

卷积层里的填充和步幅
- 卷积层控制输出大小的两个超参数：一个叫填充，一个叫步幅。
- 为什么我们需要填充？假设给定一个32x32的图像，应用5x5大小的卷积核，第一层得到输出大小28x28，第7层得到输出大小4x4，更大的卷积核可以更快地减小输出大小，形状从n_h x n_w减少到(n_h - k_h + 1) x (n_w - k_w + 1)。会有个问题，假设不想输出变得这么小，或者说我们想用很多层的时候，因为现在的输出太小了，就无法再用了。那么如果我想用更深的神经网络怎么办？因为我们整个深度学习就是希望用更深的模型，例如几百层的模型。

填充 padding
- 在输入周围添加额外的行/列
- 填充p_h行和p_w列，输出形状为 (n_h - k_h + p_h + 1) x (n_w - k_w + p_w + 1)。
- 通常取p_h = k_h - 1, p_w = k_w - 1。通常会填充核的高宽减一的值。好处是输入输出形状不会发生变化。
- 当k_h为奇数：在上下两侧填充p_h/2；
- 当k_h为偶数：在上侧填充p_h/2向上取整，在下侧填充p_h/2向下取整。很少会用偶数的卷积核。


步幅 stride
- 填充减小的输出大小与层数线性相关
- 给定输入大小224x224，在使用5x5卷积核的情况下，需要44层将输出降低到4x4
- 需要大量计算才能得到较小输出
- 比较深的神经网络，他通常的输入都是比较大的224x224，实际上是个很小的图片，这样44层就比较痛苦，虽然可以使用较大的卷积核，但是我们通常用5x5或3x3，那么就需要大量的计算才能得到较小的输出。
- 步幅是来解决这个问题。输出的大小是跟层数线性相关的，步幅可以让他变成指数相关。
- 之前移动窗口的时候都是往右或往下移一格，步幅就是每次可以不要移一格，而是移两格。
- 步幅是指行/列的滑动步长。例：高度3宽度2的步幅。
- 给定高度s_h和宽度s_w的步幅，输出形状是 (n_h - k_h + p_h + s_h)/s_h 向下取整 X (n_w - k_w + p_w + s_w)/s_w 向下取整。
- 如果 p_h = k_h - 1, p_w = k_w - 1, 那么(n_h + s_h - 1)/s_h 向下取整 X (n_w + s_w - 1)/s_w 向下取整。
- 如果输入高度和宽度可以被步幅整除，那么 (n_h/s_h) X (n_w/s_w)。步幅通常取2，n_h和n_w取偶数。

总结
- 填充和步幅是卷积层的超参数
- 填充在输入周围添加额外的行/列，通常为0，来控制输出形状的减少量，让输出形状不变或者变大
- 步幅是每次滑动核窗口时的行/列的步长，可以成倍的减少输出形状。当输入的大小比较大的时候，可以减少计算量

- 一般来说，填充会使得输入和输出一样，填充通常是取(核-1)，这样纯粹是算起来比较方便。
- 通常来讲，步幅等于1是最好的，如果计算量太大，步幅通常取2，每次减半。什么时候取2看复杂度，例如224输入，需要100层，就把5个步幅是2的神经网络均匀地插在神经网络中间。
- 核大小通常是最关键的。一般来说，卷积核就是3x3.
- 一般来说不会真的手写神经网络，而是用ResNet系列。很多时候网络结构没有那么关键。取决于数据怎么做预处理等等。
- 为什么用3x3的卷积核？视野没问题，我需要用比较深的神经网络，使得最后一层每一个元素能看到足够多的图片信息。
- NAS可以让超参数一起训练。
- 从信息论的角度来看信息总是会丢失的。机器学习本质上是压缩，所有机器学习算法永远是会丢失信息的。
- 验证集选的比较好的话，可以很好的控制过拟合。
- 虽然10层3x3等价于5-6层5x5的卷积，但是3x3训练会快一些，因为计算复杂度跟高宽相关。所以用小的卷积会快一些，这也是GoogleNet最核心的设计思路。
- 主流的做法是，底层用相对大一点的kernel，例如5x5, 7x7, 甚至11x11的kernel，但是之后都用3x3。如果效果不明显就用一样的。
- 深度学习很流行，所以他不是有钱人的游戏。深度学习让GPU替代掉了很多人力成本和数据成本。NAS现在是有钱人的游戏。







In [ ]:
# 填充与步幅
# 在所有侧边填充1个像素
import torch
from torch import nn

# 为了方便起见，我们定义了一个计算卷积层的函数
# 此函数初始化卷积层权重，并对输入和输出提高和缩减相应的位数
def comp_conv2d(conv2d, X):
    # 在维度的前面增加一个通道数核批量大小数
    # 这里的(1, 1)表示批量大小和通道数都是1
    X = X.reshape((1, 1) + X.shape)
    Y = conv2d(X)
    # 这是4维的，把前面两维拿掉
    return Y.reshape(Y.shape[2:])

# 请注意，这里每边都填充了1行或1列，因此总共添加了2行或2列
# 输入输出的通道数都是为1，核的大小为3，填充为1。之前讲的填充是左右两边一共填充的数字，对于框架来说，填充是指上下左右一边填充的数字。
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1)
X = torch.rand(size=(8, 8))
comp_conv2d(conv2d, X).shape
# 8 + 2 - 3 + 1 = 8

torch.Size([8, 8])

In [ ]:
# 填充不同的高度和宽度
conv2d = nn.Conv2d(1, 1, kernel_size=(5, 3), padding=(2, 1))
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

In [ ]:
# 将高度和宽度的步幅设置为2
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1, stride=2)
comp_conv2d(conv2d, X).shape

torch.Size([4, 4])

In [ ]:
# (8 + 2 - 5 + 4) / 4 = 2
conv2d = nn.Conv2d(1, 1, kernel_size=(3, 5), padding=(0, 1), stride=(3, 4))
comp_conv2d(conv2d, X).shape

torch.Size([2, 2])

In [ ]:
# 多输入多输出通道 channels
import torch
from d2l import torch as d2l

def corr2d_multi_in(X, K):
    # 先遍历